# Phase 1.2 — Preprocessing & Normalization

**Objective:** Transform the three aligned Parquet files produced by Phase 1.1
(`toniot_aligned.parquet`, `iot23_complete.parquet`, `botiot_52col.parquet`) into
mathematically normalized, stratified, and encoded numpy tensors ready for Neural
Network training.

---

## Pipeline

| Step | Operation | Fit on | Apply to |
|---|---|---|---|
| 1 | **Stratified Subsampling** | — | All 3 Parquets |
| 2 | **Train / Val / Test Split** | — | Subsample |
| 3 | **Sentinel Imputation** | Train | Train, Val |
| 4a | **Log1p + StandardScaler** | Train | Train, Val |
| 4b | **OneHotEncoder** (proto, state) | Train | Train, Val |
| 4c | **Frequency Encoding** (ports) | Train | Train, Val |
| 5 | **Export `.npy` tensors + `preprocessors.pkl`** | — | — |

## Key Constraints
- **Memory safe:** Input is ~351M rows; must subsample before loading into RAM.
- **No leakage:** All statistics (medians, means, stds, frequencies, encoders) are
  fitted *exclusively* on the Training set.
- **Rare class preservation:** Classes with fewer rows than the cap keep 100% of
  their samples — never downsampled.
- **Sentinel handling:** Group B sentinel `-1`/`-1.0` → `NaN` → median impute
  (train median applied to val/test). Categorical `<absent>` treated as a valid
  learnable token and handled by OneHotEncoder `handle_unknown='ignore'`.

In [1]:
# ─────────────────────────────────────────────────────────────────────────────
# CELL 1 | Imports + Dependency Check
# ─────────────────────────────────────────────────────────────────────────────
import sys, os, json, time, pickle, warnings
from pathlib import Path

import numpy as np
import pandas as pd

warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning)

# ── PyArrow ───────────────────────────────────────────────────────────────────
try:
    import pyarrow as pa
    import pyarrow.parquet as pq
    print(f"✅ pyarrow {pa.__version__}")
except ImportError:
    raise SystemExit("pyarrow not found — run: pip install pyarrow")

# ── scikit-learn ──────────────────────────────────────────────────────────────
try:
    from sklearn.preprocessing import StandardScaler, OneHotEncoder
    from sklearn.model_selection import train_test_split
    import sklearn
    print(f"✅ scikit-learn {sklearn.__version__}")
except ImportError:
    raise SystemExit("scikit-learn not found — run: pip install scikit-learn")

# ── tqdm ──────────────────────────────────────────────────────────────────────
try:
    from tqdm import tqdm
    print("✅ tqdm available")
except ImportError:
    def tqdm(iterable, **kwargs):  # silent fallback
        return iterable
    print("ℹ️  tqdm not found — progress bars disabled")

print(f"\n✅ Python {sys.version.split()[0]}")
print(f"✅ pandas  {pd.__version__}")
print(f"✅ numpy   {np.__version__}")

✅ pyarrow 23.0.0
✅ scikit-learn 1.7.0
✅ tqdm available

✅ Python 3.13.9
✅ pandas  2.2.3
✅ numpy   2.1.3


In [2]:
# ─────────────────────────────────────────────────────────────────────────────
# CELL 2 | Paths + Constants
# ─────────────────────────────────────────────────────────────────────────────
NOTEBOOK_DIR  = Path.cwd()              # .../Phase_1/
MAIN_DIR      = NOTEBOOK_DIR.parent    # .../main_folder/
UNIFIED_DIR   = MAIN_DIR / "data" / "unified"
PROCESSED_DIR = MAIN_DIR / "data" / "processed"
ARTIFACTS_DIR = MAIN_DIR / "artifacts"

PROCESSED_DIR.mkdir(parents=True, exist_ok=True)
ARTIFACTS_DIR.mkdir(parents=True, exist_ok=True)

# ── Parquet inputs (produced by Phase 1.1) ────────────────────────────────────
PARQUET_FILES = {
    "toniot": UNIFIED_DIR / "toniot_aligned.parquet",
    "iot23":  UNIFIED_DIR / "iot23_complete.parquet",
    "botiot": UNIFIED_DIR / "botiot_52col.parquet",
}

# ── Output paths ──────────────────────────────────────────────────────────────
X_TRAIN_PATH        = PROCESSED_DIR / "X_train.npy"
Y_TRAIN_PATH        = PROCESSED_DIR / "y_train.npy"
X_VAL_PATH          = PROCESSED_DIR / "X_val.npy"
Y_VAL_PATH          = PROCESSED_DIR / "y_val.npy"
X_TEST_RAW_PATH     = PROCESSED_DIR / "X_test_raw.parquet"   # raw, unscaled — saved early
PREPROCESSORS_PATH  = ARTIFACTS_DIR / "preprocessors.pkl"
SUBSAMPLE_PATH      = PROCESSED_DIR / "subsample.parquet"           # written by run_subsample.py
SUBSAMPLE_META_PATH = PROCESSED_DIR / "subsample_metadata.parquet"  # IPs + timestamps for RAG

# ── Column Schema (from Phase 1.1 FINAL_COLUMNS) ─────────────────────────────
# Group A numerical features (log1p → StandardScaler)
NUM_COLS = [
    "univ_duration", "univ_src_bytes", "univ_dst_bytes",
    "univ_src_pkts",  "univ_dst_pkts",
]

# Port columns (frequency encoding)
PORT_COLS = ["univ_src_port", "univ_dst_port"]

# Boolean columns (keep as int8 — no scaling)
BOOL_COLS = ["univ_has_src_port", "univ_has_dst_port"]

# Categorical columns (OneHotEncoder)
CAT_COLS = ["univ_proto", "univ_state"]

# Group B numerical columns with sentinel -1 / -1.0 (impute then scale)
SENTINEL_NUM_COLS = [
    # Zeek shared
    "zeek_missed_bytes", "zeek_src_ip_bytes", "zeek_dst_ip_bytes",
    # TON-IoT only
    "toniot_dns_qclass", "toniot_dns_qtype", "toniot_dns_rcode",
    "toniot_http_request_body_len", "toniot_http_response_body_len",
    "toniot_http_status_code",
    # Bot-IoT behavioral windows
    "botiot_mean", "botiot_stddev", "botiot_sum", "botiot_min",
    "botiot_max",  "botiot_rate",   "botiot_srate", "botiot_drate",
    "botiot_TnBPSrcIP", "botiot_TnBPDstIP",
    "botiot_TnP_PSrcIP", "botiot_TnP_PDstIP",
    "botiot_TnP_PerProto", "botiot_TnP_Per_Dport",
    "botiot_AR_P_Proto_P_SrcIP", "botiot_AR_P_Proto_P_DstIP",
    "botiot_N_IN_Conn_P_DstIP",  "botiot_N_IN_Conn_P_SrcIP",
    "botiot_AR_P_Proto_P_Sport", "botiot_AR_P_Proto_P_Dport",
    "botiot_Pkts_P_State_P_Protocol_P_DestIP",
    "botiot_Pkts_P_State_P_Protocol_P_SrcIP",
]

# Group B categorical sentinels — treat <absent> as a valid token (no imputation)
SENTINEL_CAT_COLS = ["zeek_service", "zeek_history"]

# Metadata columns — saved separately for RAG, NOT included in X tensors
META_COLS = ["dataset_source", "meta_src_ip", "meta_dst_ip", "meta_timestamp"]

# Label columns
LABEL_BINARY     = "univ_label_binary"
LABEL_MULTICLASS = "univ_label_multiclass"
LABEL_STR        = "univ_label_str"
ATTACK_COL       = "univ_specific_attack"

# 5-class taxonomy
LABEL_CLASS_NAMES = {
    0: "Normal",
    1: "Reconnaissance",
    2: "Volumetric_Attack",
    3: "C2_Botnet",
    4: "Exploit_and_Theft",
}

# ── Verify inputs exist ───────────────────────────────────────────────────────
print("=" * 65)
print("📁 INPUT PARQUET FILES")
print("=" * 65)
all_found = True
for name, path in PARQUET_FILES.items():
    if path.exists():
        size_mb = path.stat().st_size / 1024**2
        nrows   = pq.ParquetFile(str(path)).metadata.num_rows
        print(f"  ✅  {name:<8s}  {size_mb:>7.1f} MB   {nrows:>15,} rows   {path.name}")
    else:
        print(f"  ❌  {name:<8s}  MISSING → {path}")
        all_found = False

if not all_found:
    raise FileNotFoundError("One or more Parquet files missing. Run Phase 1.1 first.")

print(f"\n✅ All inputs found — output dir: {PROCESSED_DIR}")

📁 INPUT PARQUET FILES
  ✅  toniot      258.1 MB        22,339,021 rows   toniot_aligned.parquet
  ✅  iot23      5830.1 MB       325,309,946 rows   iot23_complete.parquet
  ✅  botiot       82.3 MB         3,668,522 rows   botiot_52col.parquet

✅ All inputs found — output dir: c:\Users\suhas\OneDrive\Desktop\Capstone\RAG-IDS-Knowledge-Augmented-IoT-Threat-Detection\main_folder\data\processed


---
## Step 1 — Stratified Subsampling

**Problem:** 351M rows cannot fit in RAM.  
**Solution:** Stream each Parquet in row-group chunks, greedily collecting rows per class
until each class's target cap is reached.

**Cap policy:**
- Majority classes (Normal, Reconnaissance) → capped at `500,000` to prevent imbalance.
- Minority classes (C2_Botnet, Volumetric_Attack) → capped at their actual count — i.e.,
  100% kept if they don't exceed the cap.
- Rarest class (Exploit_and_Theft) → **always 100%** of available samples.

Result: ~2–3M balanced rows in a single DataFrame.

In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# CELL 3 | Stratified Subsampling — Function
# ─────────────────────────────────────────────────────────────────────────────

# Target row cap per class (set to None = keep all rows of that class)
# ⚠️  HONEST CAPS — must match run_subsample.py exactly for reproducibility
CLASS_CAPS = {
    0: 500_000,    # Normal             — large majority, cap to prevent dominance
    1: 500_000,    # Reconnaissance     — large majority, cap to prevent dominance
    2: 500_000,    # Volumetric_Attack  — keep up to 500k
    3: 2_000_000,  # C2_Botnet          — CAPPED (true total is ~61M from IoT-23;
                   #                       keeping all requires 50+ GB RAM)
    4: None,       # Exploit_and_Theft  — keep 100% (~4.86M rows, feasible in RAM)
}

def stratified_subsample(
    parquet_files: dict,
    class_caps: dict,
    label_col: str = LABEL_MULTICLASS,
    seed: int = 42,
) -> pd.DataFrame:
    """
    Stream all Parquet files row-group by row-group, collecting rows per class
    until each class cap is reached.

    Parameters
    ----------
    parquet_files : dict  {name: Path}  — Phase 1.1 outputs
    class_caps    : dict  {class_id: int | None}  — None = keep all
    label_col     : str   — column holding the integer class label
    seed          : int   — for reproducible row-group shuffling within a class

    Returns
    -------
    pd.DataFrame  — concatenated subsample, shuffled
    """
    rng = np.random.default_rng(seed)

    # Buckets: class_id → list of DataFrames
    buckets: dict[int, list] = {cls: [] for cls in class_caps}
    counts:  dict[int, int]  = {cls: 0   for cls in class_caps}

    # True if a class is completely filled (cap reached)
    def is_full(cls):
        cap = class_caps[cls]
        return cap is not None and counts[cls] >= cap

    for ds_name, pq_path in parquet_files.items():
        pf     = pq.ParquetFile(str(pq_path))
        n_rg   = pf.metadata.num_row_groups
        print(f"\n  ▶  {ds_name:<8s}  ({n_rg} row-groups)")

        # Check whether ALL classes from this file are already full → skip
        if all(is_full(c) for c in class_caps):
            print("       All class caps already reached — skipping file.")
            continue

        rg_order = rng.permutation(n_rg)          # randomise row-group order

        for rg_idx in tqdm(rg_order, desc=f"  {ds_name}", leave=False):
            if all(is_full(c) for c in class_caps):
                break

            # Read only the label column first to check which classes are present
            rg_labels = (
                pf.read_row_group(rg_idx, columns=[label_col])
                  .to_pandas()[label_col]
            )

            # Determine which classes in this row-group still need more rows
            needed_classes = {c for c in class_caps if not is_full(c)}
            has_needed = rg_labels.isin(needed_classes).any()
            if not has_needed:
                continue

            # Read the full row-group
            rg_df = pf.read_row_group(rg_idx).to_pandas()

            for cls in needed_classes:
                cap      = class_caps[cls]
                cls_rows = rg_df[rg_df[label_col] == cls]
                if cls_rows.empty:
                    continue

                if cap is not None:
                    remaining = cap - counts[cls]
                    if len(cls_rows) > remaining:
                        cls_rows = cls_rows.sample(remaining, random_state=seed)

                buckets[cls].append(cls_rows)
                counts[cls] += len(cls_rows)

        # Status after each file
        for cls, cnt in counts.items():
            cap_str = str(class_caps[cls]) if class_caps[cls] else "ALL"
            print(f"     Class {cls} {LABEL_CLASS_NAMES[cls]:<20s}: {cnt:>9,} / {cap_str}")

    # Concatenate all buckets
    print("\n  🔗 Concatenating buckets …")
    frames = []
    for cls in class_caps:
        if buckets[cls]:
            frames.append(pd.concat(buckets[cls], ignore_index=True))

    df = pd.concat(frames, ignore_index=True)

    # Global shuffle — ignore_index avoids an internal full-array copy
    df = df.sample(frac=1, random_state=seed, ignore_index=True)

    return df


print("✅ stratified_subsample() defined")
print(f"\nClass caps (must match run_subsample.py):")
for cls, cap in CLASS_CAPS.items():
    cap_display = f"{cap:,}" if cap else "ALL (keep 100%)"
    print(f"  Class {cls} {LABEL_CLASS_NAMES[cls]:<20s}: {cap_display}")


✅ stratified_subsample() defined

Class caps:
  Class 0 Normal              : 500000
  Class 1 Reconnaissance      : 500000
  Class 2 Volumetric_Attack   : 500000
  Class 3 C2_Botnet           : ALL (keep 100%)
  Class 4 Exploit_and_Theft   : ALL (keep 100%)


In [14]:
# ─────────────────────────────────────────────────────────────────────────────
# CELL 4 | Execute Stratified Subsampling
# ─────────────────────────────────────────────────────────────────────────────
# ⚠️  This step streams 351M rows and is MEMORY INTENSIVE.
#
# RECOMMENDED: Run the standalone script FIRST (outside VS Code) to avoid
# kernel OOM crashes:
#
#   python main_folder/scripts/run_subsample.py
#
# Once run_subsample.py has written  data/processed/subsample.parquet,
# this cell will load from that file instantly instead of re-streaming.
# ─────────────────────────────────────────────────────────────────────────────

print("=" * 65)
print("🎲  STEP 1 — STRATIFIED SUBSAMPLING")
print("=" * 65)

if SUBSAMPLE_PATH.exists():
    # ── Fast path: pre-built subsample exists ─────────────────────────────────
    size_mb = SUBSAMPLE_PATH.stat().st_size / 1024**2
    print(f"  ✅ Found pre-built subsample ({size_mb:.1f} MB) — loading from disk …")
    t0 = time.time()
    df_sample = pd.read_parquet(SUBSAMPLE_PATH, engine="pyarrow")
    print(f"  ⏱  Loaded in {time.time()-t0:.1f}s")
else:
    # ── Slow path: run in-process (may crash VS Code on large machines) ───────
    print("  ⚠️  subsample.parquet not found — running in-process subsampling.")
    print("  ℹ️  If VS Code crashes, run:  python main_folder/scripts/run_subsample.py")
    print("       then re-run this cell.\n")
    t0 = time.time()
    df_sample = stratified_subsample(
        parquet_files = PARQUET_FILES,
        class_caps    = CLASS_CAPS,
        label_col     = LABEL_MULTICLASS,
        seed          = 42,
    )
    # Save for future runs
    print("\n  💾 Saving subsample for future runs …")
    df_sample.to_parquet(SUBSAMPLE_PATH, index=False, engine="pyarrow")
    print(f"  ✅ Saved → {SUBSAMPLE_PATH.name}")
    print(f"  ⏱  Total time: {time.time()-t0:.1f}s")

print(f"\n  Total rows   : {len(df_sample):,}")
print(f"  Columns      : {len(df_sample.columns)}")
print(f"  RAM usage    : ~{df_sample.memory_usage(deep=True).sum() / 1024**2:.0f} MB")

print(f"\n{'─'*65}")
print("Class distribution in subsample:")
dist = df_sample[LABEL_MULTICLASS].value_counts().sort_index()
total = len(df_sample)
for cls_id, cnt in dist.items():
    print(f"  Class {cls_id} {LABEL_CLASS_NAMES.get(cls_id, '?'):<20s}: {cnt:>9,}  ({cnt/total*100:.2f}%)")


🎲  STEP 1 — STRATIFIED SUBSAMPLING
  ✅ Found pre-built subsample (251.2 MB) — loading from disk …
  ⏱  Loaded in 4.5s

  Total rows   : 8,362,241
  Columns      : 52
  RAM usage    : ~6390 MB

─────────────────────────────────────────────────────────────────
Class distribution in subsample:
  Class 0 Normal              :   500,000  (5.98%)
  Class 1 Reconnaissance      :   500,000  (5.98%)
  Class 2 Volumetric_Attack   :   500,000  (5.98%)
  Class 3 C2_Botnet           : 2,000,000  (23.92%)
  Class 4 Exploit_and_Theft   : 4,862,241  (58.15%)


---
## Step 2 — Train / Val / Test Split

**Stratified 80 / 10 / 10 split** — same class proportions in every split.

The **Test set is saved immediately as raw Parquet** and removed from memory.
No statistics from Test can leak into the preprocessing pipeline.

In [15]:
# ─────────────────────────────────────────────────────────────────────────────
# CELL 5 | Stratified Train / Val / Test Split
# ─────────────────────────────────────────────────────────────────────────────
print("=" * 65)
print("✂️   STEP 2 — STRATIFIED SPLIT  (80 / 10 / 10)")
print("=" * 65)

SPLIT_SEED   = 42
TEST_SIZE    = 0.10
VAL_SIZE     = 0.10   # fraction of the non-test portion → 10% of total

# ── First split: train+val vs test ────────────────────────────────────────────
df_trainval, df_test = train_test_split(
    df_sample,
    test_size    = TEST_SIZE,
    stratify     = df_sample[LABEL_MULTICLASS],
    random_state = SPLIT_SEED,
)

# ── Save test set immediately (raw, unscaled) — then drop from RAM ─────────────
print(f"  Saving raw test set ({len(df_test):,} rows) → {X_TEST_RAW_PATH.name}")
df_test.to_parquet(X_TEST_RAW_PATH, index=False, engine="pyarrow")
del df_test   # ← prevent leakage
print("  ✅ Test set saved and removed from memory")

# ── Second split: train vs val ────────────────────────────────────────────────
# val_size relative to trainval = 10/90 ≈ 0.1111  → gives 10% of total
val_ratio = VAL_SIZE / (1 - TEST_SIZE)
df_train, df_val = train_test_split(
    df_trainval,
    test_size    = val_ratio,
    stratify     = df_trainval[LABEL_MULTICLASS],
    random_state = SPLIT_SEED,
)
del df_trainval
df_train = df_train.reset_index(drop=True)
df_val   = df_val.reset_index(drop=True)

print(f"\n  Train rows   : {len(df_train):,}  (≈80%)")
print(f"  Val   rows   : {len(df_val):,}  (≈10%)")
print(f"  Test  rows   : saved to parquet → not in memory")

# ── Detach metadata now (save separately for RAG) ─────────────────────────────
train_meta = df_train[META_COLS + [LABEL_MULTICLASS, LABEL_STR, ATTACK_COL]].copy()
val_meta   = df_val[META_COLS   + [LABEL_MULTICLASS, LABEL_STR, ATTACK_COL]].copy()

train_meta.to_parquet(PROCESSED_DIR / "train_meta.parquet", index=False)
val_meta.to_parquet(PROCESSED_DIR   / "val_meta.parquet",   index=False)
print("\n  ✅ RAG metadata saved: train_meta.parquet, val_meta.parquet")

# Drop meta + string label cols from working frames (keep only features + int labels)
DROP_FROM_X = META_COLS + [LABEL_STR, ATTACK_COL, LABEL_BINARY]
df_train.drop(columns=DROP_FROM_X, inplace=True, errors="ignore")
df_val.drop(  columns=DROP_FROM_X, inplace=True, errors="ignore")

print(f"\n  Working feature columns : {len(df_train.columns) - 1}  (+1 label)")

# ── Verify stratification ─────────────────────────────────────────────────────
print(f"\n{'─'*65}")
print("  Class proportions (should be ~equal across splits):")
print(f"  {'Class':<5s} {'Name':<22s} {'Train':>8s} {'Val':>8s}")
for cls in sorted(LABEL_CLASS_NAMES):
    tr_cnt = (df_train[LABEL_MULTICLASS] == cls).sum()
    vl_cnt = (df_val[LABEL_MULTICLASS]   == cls).sum()
    print(f"  {cls:<5d} {LABEL_CLASS_NAMES[cls]:<22s} {tr_cnt:>8,} {vl_cnt:>8,}")

✂️   STEP 2 — STRATIFIED SPLIT  (80 / 10 / 10)
  Saving raw test set (836,225 rows) → X_test_raw.parquet
  ✅ Test set saved and removed from memory

  Train rows   : 6,689,792  (≈80%)
  Val   rows   : 836,224  (≈10%)
  Test  rows   : saved to parquet → not in memory

  ✅ RAG metadata saved: train_meta.parquet, val_meta.parquet

  Working feature columns : 44  (+1 label)

─────────────────────────────────────────────────────────────────
  Class proportions (should be ~equal across splits):
  Class Name                      Train      Val
  0     Normal                  400,000   50,000
  1     Reconnaissance          400,000   50,000
  2     Volumetric_Attack       400,000   50,000
  3     C2_Botnet              1,600,000  200,000
  4     Exploit_and_Theft      3,889,792  486,224


---
## Step 3 — Sentinel Imputation

Group B numerical columns carry `-1` / `-1.0` as a sentinel meaning
"this column does not exist for this dataset".

**Strategy:**
1. Replace `-1`/`-1.0` with `NaN`.
2. Calculate the **median** of valid (non-NaN) values **on the Training set only**.
3. Fill NaN in Train with its own median; fill NaN in Val with the **same training median**.

In [16]:
# ─────────────────────────────────────────────────────────────────────────────
# CELL 6 | Sentinel Imputation (Fit on Train)
# ─────────────────────────────────────────────────────────────────────────────
print("=" * 65)
print("🔧  STEP 3 — SENTINEL IMPUTATION")
print("=" * 65)

SENTINEL_INT   = -1
SENTINEL_FLOAT = -1.0

imputer_stats: dict[str, float] = {}

print("  Fitting medians on Training set …")

for col in tqdm(SENTINEL_NUM_COLS, desc="  Impute-fit"):
    if col not in df_train.columns:
        continue

    # Replace both sentinel forms with NaN
    train_col = df_train[col].astype(float).replace(
        {float(SENTINEL_INT): np.nan, SENTINEL_FLOAT: np.nan}
    )

    # Calculate median of valid values (Training only)
    median_val = float(train_col.median())   # NaN-safe
    if np.isnan(median_val):
        median_val = 0.0    # fallback: all values were sentinel → impute 0

    imputer_stats[col] = median_val

    # Fill Training set
    df_train[col] = train_col.fillna(median_val).astype(float)


def apply_imputation(df: pd.DataFrame, stats: dict) -> pd.DataFrame:
    """Apply pre-fitted imputation medians to a DataFrame (val / test)."""
    df = df.copy()
    for col, median_val in stats.items():
        if col not in df.columns:
            continue
        df[col] = (
            df[col].astype(float)
                   .replace({float(SENTINEL_INT): np.nan, SENTINEL_FLOAT: np.nan})
                   .fillna(median_val)
        )
    return df


df_val = apply_imputation(df_val, imputer_stats)

print(f"  ✅ Fitted and applied imputation for {len(imputer_stats)} columns")
print(f"\n  Sample imputer medians:")
sample_cols = list(imputer_stats.items())[:8]
for col, med in sample_cols:
    print(f"    {col:<50s}: {med:.4f}")
if len(imputer_stats) > 8:
    print(f"    ... and {len(imputer_stats) - 8} more")

# Verify no -1 sentinels remain in sentinel columns
sentinel_remaining_train = 0
for col in SENTINEL_NUM_COLS:
    if col in df_train.columns:
        sentinel_remaining_train += (df_train[col] == SENTINEL_FLOAT).sum()
print(f"\n  ✅ Sentinel -1 remaining in Train: {sentinel_remaining_train}  (should be 0)")

🔧  STEP 3 — SENTINEL IMPUTATION
  Fitting medians on Training set …


  Impute-fit: 100%|██████████| 31/31 [00:10<00:00,  3.03it/s]


  ✅ Fitted and applied imputation for 31 columns

  Sample imputer medians:
    zeek_missed_bytes                                 : 0.0000
    zeek_src_ip_bytes                                 : 130.0000
    zeek_dst_ip_bytes                                 : 132.0000
    toniot_dns_qclass                                 : 0.0000
    toniot_dns_qtype                                  : 0.0000
    toniot_dns_rcode                                  : 0.0000
    toniot_http_request_body_len                      : 0.0000
    toniot_http_response_body_len                     : 0.0000
    ... and 23 more

  ✅ Sentinel -1 remaining in Train: 0  (should be 0)


---
## Step 4 — Feature Engineering

### 4a. Numerical: `log1p` → `StandardScaler`
Network traffic features are heavy-tailed (src_bytes can span 0–10⁹).
`log1p` compresses extreme values before Z-score normalization.

### 4b. Categorical: `OneHotEncoder`
Applied to `univ_proto` (5 tokens) and `univ_state` (5 tokens).
`<absent>` is a valid vocab token and will produce its own OHE column.

### 4c. Port Frequency Encoding
Port numbers range 0–65535. Replacing each port with its frequency (probability
of occurrence in Training) gives a single continuous feature that captures
how "common" the port is — without inflating dimensionality.

In [17]:
# ─────────────────────────────────────────────────────────────────────────────
# CELL 7 | Feature Engineering 4a — log1p + StandardScaler (Numerical)
# ─────────────────────────────────────────────────────────────────────────────
print("=" * 65)
print("📐  STEP 4a — NUMERICAL: log1p + StandardScaler")
print("=" * 65)

# Columns to scale: Group A numericals + Group B sentinel-imputed numericals
ALL_NUM_COLS = NUM_COLS + SENTINEL_NUM_COLS
ALL_NUM_COLS = [c for c in ALL_NUM_COLS if c in df_train.columns]

# ── log1p transform — clip negatives to 0 first (sentinel cols already imputed) ─
for col in tqdm(ALL_NUM_COLS, desc="  log1p"):
    df_train[col] = np.log1p(np.maximum(df_train[col].astype(float), 0.0))
    df_val[col]   = np.log1p(np.maximum(df_val[col].astype(float),   0.0))

# ── StandardScaler — fit on Train ─────────────────────────────────────────────
scaler = StandardScaler()
df_train[ALL_NUM_COLS] = scaler.fit_transform(df_train[ALL_NUM_COLS].astype(float))
df_val[ALL_NUM_COLS]   = scaler.transform(df_val[ALL_NUM_COLS].astype(float))

print(f"  Scaled {len(ALL_NUM_COLS)} numerical columns")
print(f"  Train mean (first 3 cols): "
      f"{df_train[ALL_NUM_COLS[:3]].mean().round(4).to_dict()}")
print(f"  Train std  (first 3 cols): "
      f"{df_train[ALL_NUM_COLS[:3]].std().round(4).to_dict()}")
print("  ✅ StandardScaler fitted on Train, applied to Val")

📐  STEP 4a — NUMERICAL: log1p + StandardScaler


  log1p: 100%|██████████| 36/36 [00:05<00:00,  7.15it/s]


  Scaled 36 numerical columns
  Train mean (first 3 cols): {'univ_duration': -0.0, 'univ_src_bytes': -0.0, 'univ_dst_bytes': -0.0}
  Train std  (first 3 cols): {'univ_duration': 1.0, 'univ_src_bytes': 1.0, 'univ_dst_bytes': 1.0}
  ✅ StandardScaler fitted on Train, applied to Val


In [18]:
# ─────────────────────────────────────────────────────────────────────────────
# CELL 8 | Feature Engineering 4b — OneHotEncoder (Categorical)
# ─────────────────────────────────────────────────────────────────────────────
print("=" * 65)
print("🔡  STEP 4b — CATEGORICAL: OneHotEncoder")
print("=" * 65)

# ── Source-identity leak guard ────────────────────────────────────────────────
# SENTINEL_CAT_COLS = ["zeek_service", "zeek_history"]
#
# These columns are NOT included in OHE. Here is why:
#   - Bot-IoT has no Zeek data  → both columns are "<absent>"
#   - TON-IoT has no zeek_history → it is "<absent>"
#   - IoT-23 has genuine Zeek values
#
# If we OHE "<absent>", the model gets a feature "zeek_history_<absent>"
# which == 1 for every non-IoT-23 row. The NN would learn dataset source
# as a proxy rather than true attack patterns — a form of data leakage.
#
# Fix: drop both Zeek sentinel categoricals from the feature matrix entirely.
# ─────────────────────────────────────────────────────────────────────────────

# Drop zeek sentinel categoricals before OHE (source-identity leak prevention)
ZEEK_SOURCE_COLS = [c for c in SENTINEL_CAT_COLS if c in df_train.columns]
if ZEEK_SOURCE_COLS:
    df_train.drop(columns=ZEEK_SOURCE_COLS, inplace=True)
    df_val.drop(  columns=ZEEK_SOURCE_COLS, inplace=True)
    print(f"  🗑️  Dropped source-identity columns (not encoded): {ZEEK_SOURCE_COLS}")

# Only OHE true protocol/state categoricals
ALL_CAT_COLS = [c for c in CAT_COLS if c in df_train.columns]
print(f"  Encoding columns: {ALL_CAT_COLS}")

# Fill any NaN in categorical columns with '<absent>'
for col in ALL_CAT_COLS:
    df_train[col] = df_train[col].fillna("<absent>").astype(str)
    df_val[col]   = df_val[col].fillna("<absent>").astype(str)

ohe = OneHotEncoder(
    handle_unknown = "ignore",    # unseen tokens in val → all-zero row
    sparse_output  = False,       # dense numpy array
    dtype          = np.float32,
)

# Fit on Train
ohe_train = ohe.fit_transform(df_train[ALL_CAT_COLS])
ohe_val   = ohe.transform(df_val[ALL_CAT_COLS])

ohe_feature_names = ohe.get_feature_names_out(ALL_CAT_COLS)

print(f"  OHE output shape  — Train: {ohe_train.shape}, Val: {ohe_val.shape}")
print(f"  Generated {len(ohe_feature_names)} OHE features:")
for name in ohe_feature_names:
    print(f"    • {name}")

# Drop original categorical columns from DataFrames
df_train.drop(columns=ALL_CAT_COLS, inplace=True)
df_val.drop(  columns=ALL_CAT_COLS, inplace=True)

print("  ✅ OneHotEncoder fitted on Train, applied to Val")
print("  ✅ Source-identity columns excluded — model must learn attacks, not dataset origin")


🔡  STEP 4b — CATEGORICAL: OneHotEncoder
  🗑️  Dropped source-identity columns (not encoded): ['zeek_service', 'zeek_history']
  Encoding columns: ['univ_proto', 'univ_state']
  OHE output shape  — Train: (6689792, 8), Val: (836224, 8)
  Generated 8 OHE features:
    • univ_proto_arp
    • univ_proto_icmp
    • univ_proto_tcp
    • univ_proto_udp
    • univ_state_attempt
    • univ_state_established
    • univ_state_other
    • univ_state_rejected
  ✅ OneHotEncoder fitted on Train, applied to Val
  ✅ Source-identity columns excluded — model must learn attacks, not dataset origin


In [19]:
# ─────────────────────────────────────────────────────────────────────────────
# CELL 9 | Feature Engineering 4c — Port Frequency Encoding
# ─────────────────────────────────────────────────────────────────────────────
print("=" * 65)
print("🔢  STEP 4c — PORT FREQUENCY ENCODING")
print("=" * 65)

port_freq_maps: dict[str, dict] = {}

for col in PORT_COLS:
    if col not in df_train.columns:
        continue

    # Compute frequency (probability) of each port value in Training set
    freq = df_train[col].value_counts(normalize=True).to_dict()
    port_freq_maps[col] = freq

    # Map train ports → frequency; unknown ports → 0.0
    df_train[col] = df_train[col].map(freq).fillna(0.0).astype(np.float32)
    df_val[col]   = df_val[col].map(freq).fillna(0.0).astype(np.float32)

    n_unique = len(freq)
    print(f"  {col:<20s}: {n_unique:,} unique ports in Train → encoded as frequency")

print("  ✅ Frequency encoding applied — unseen Val ports mapped to 0.0")

🔢  STEP 4c — PORT FREQUENCY ENCODING
  univ_src_port       : 65,283 unique ports in Train → encoded as frequency
  univ_dst_port       : 64,998 unique ports in Train → encoded as frequency
  ✅ Frequency encoding applied — unseen Val ports mapped to 0.0


In [20]:
# ─────────────────────────────────────────────────────────────────────────────
# CELL 10 | Assemble Final Feature Matrices
# ─────────────────────────────────────────────────────────────────────────────
print("=" * 65)
print("🧩  ASSEMBLING FINAL FEATURE MATRICES")
print("=" * 65)

# Extract labels before dropping
y_train = df_train.pop(LABEL_MULTICLASS).to_numpy(dtype=np.int8)
y_val   = df_val.pop(LABEL_MULTICLASS).to_numpy(dtype=np.int8)

# Remaining numeric columns in the DataFrames (NUM_COLS + SENTINEL_NUM_COLS + port
# + BOOL_COLS — all already scaled/encoded/frequency-encoded)
dense_feature_cols = [c for c in df_train.columns if c in df_train.columns]

X_train_dense = df_train[dense_feature_cols].to_numpy(dtype=np.float32)
X_val_dense   = df_val[dense_feature_cols].to_numpy(dtype=np.float32)

# Horizontally stack dense features + OHE columns
X_train = np.hstack([X_train_dense, ohe_train]).astype(np.float32)
X_val   = np.hstack([X_val_dense,   ohe_val  ]).astype(np.float32)

# Record feature names for later interpretability
feature_names = dense_feature_cols + list(ohe_feature_names)

print(f"  X_train : {X_train.shape}   dtype={X_train.dtype}")
print(f"  y_train : {y_train.shape}   dtype={y_train.dtype}")
print(f"  X_val   : {X_val.shape}   dtype={X_val.dtype}")
print(f"  y_val   : {y_val.shape}   dtype={y_val.dtype}")
print(f"  Feature count: {X_train.shape[1]}")

# Sanity check: no NaNs in final arrays
assert not np.isnan(X_train).any(), "❌ NaN detected in X_train!"
assert not np.isnan(X_val).any(),   "❌ NaN detected in X_val!"
assert not np.isinf(X_train).any(), "❌ Inf detected in X_train!"
print("  ✅ No NaN / Inf in X_train or X_val")

🧩  ASSEMBLING FINAL FEATURE MATRICES
  X_train : (6689792, 48)   dtype=float32
  y_train : (6689792,)   dtype=int8
  X_val   : (836224, 48)   dtype=float32
  y_val   : (836224,)   dtype=int8
  Feature count: 48
  ✅ No NaN / Inf in X_train or X_val


---
## Step 5 — Export Processed Tensors

- `X_train.npy`, `y_train.npy` — Training tensors
- `X_val.npy`, `y_val.npy` — Validation tensors
- `X_test_raw.parquet` — Raw (unscaled) test set (already saved in Step 2)
- `preprocessors.pkl` — All fitted objects (scaler, OHE, port freq maps, imputer stats)
- `feature_names.json` — Ordered list of feature names for interpretability

In [21]:
# ─────────────────────────────────────────────────────────────────────────────
# CELL 11 | Export Tensors + Preprocessors
# ─────────────────────────────────────────────────────────────────────────────
print("=" * 65)
print("💾  STEP 5 — EXPORTING TENSORS + PREPROCESSORS")
print("=" * 65)

# ── 1. Numpy tensors ──────────────────────────────────────────────────────────
t0 = time.time()
np.save(X_TRAIN_PATH, X_train)
np.save(Y_TRAIN_PATH, y_train)
np.save(X_VAL_PATH,   X_val)
np.save(Y_VAL_PATH,   y_val)
print(f"  Saved tensors in {time.time()-t0:.1f}s:")
for path, arr in [
    (X_TRAIN_PATH, X_train),
    (Y_TRAIN_PATH, y_train),
    (X_VAL_PATH,   X_val),
    (Y_VAL_PATH,   y_val),
]:
    size_mb = path.stat().st_size / 1024**2
    print(f"    ✅  {path.name:<25s}  shape={arr.shape}  {size_mb:.1f} MB")

# ── 2. Preprocessor bundle ────────────────────────────────────────────────────
preprocessors = {
    "scaler":          scaler,
    "ohe":             ohe,
    "port_freq_maps":  port_freq_maps,
    "imputer_stats":   imputer_stats,
    "all_num_cols":    ALL_NUM_COLS,
    "all_cat_cols":    ALL_CAT_COLS,
    "port_cols":       PORT_COLS,
    "bool_cols":       BOOL_COLS,
    "sentinel_num_cols": SENTINEL_NUM_COLS,
    "feature_names":   feature_names,
    "label_class_names": LABEL_CLASS_NAMES,
    "meta": {
        "phase":           "Phase_1_2_Preprocessing_Normalization",
        "n_features":      X_train.shape[1],
        "n_train":         len(y_train),
        "n_val":           len(y_val),
        "class_caps":      CLASS_CAPS,
        "split_seed":      SPLIT_SEED,
    },
}

with open(PREPROCESSORS_PATH, "wb") as fh:
    pickle.dump(preprocessors, fh, protocol=pickle.HIGHEST_PROTOCOL)
print(f"\n    ✅  {PREPROCESSORS_PATH.name:<25s}  "
      f"{PREPROCESSORS_PATH.stat().st_size/1024:.1f} KB")

# ── 3. Feature names JSON (human-readable) ────────────────────────────────────
feat_names_path = ARTIFACTS_DIR / "phase1_2_feature_names.json"
with open(feat_names_path, "w") as fh:
    json.dump({"feature_names": feature_names, "n_features": len(feature_names)}, fh, indent=2)
print(f"    ✅  {feat_names_path.name:<25s}  {len(feature_names)} features listed")

print(f"\n  Output directory: {PROCESSED_DIR}")

💾  STEP 5 — EXPORTING TENSORS + PREPROCESSORS
  Saved tensors in 2.7s:
    ✅  X_train.npy                shape=(6689792, 48)  1224.9 MB
    ✅  y_train.npy                shape=(6689792,)  6.4 MB
    ✅  X_val.npy                  shape=(836224, 48)  153.1 MB
    ✅  y_val.npy                  shape=(836224,)  0.8 MB

    ✅  preprocessors.pkl          1531.4 KB
    ✅  phase1_2_feature_names.json  48 features listed

  Output directory: c:\Users\suhas\OneDrive\Desktop\Capstone\RAG-IDS-Knowledge-Augmented-IoT-Threat-Detection\main_folder\data\processed


In [22]:
# ─────────────────────────────────────────────────────────────────────────────
# CELL 12 | Final Verification
# ─────────────────────────────────────────────────────────────────────────────
print("=" * 70)
print("🔬  FINAL VERIFICATION")
print("=" * 70)

# Reload from disk to confirm round-trip integrity
X_train_check = np.load(X_TRAIN_PATH)
y_train_check = np.load(Y_TRAIN_PATH)
X_val_check   = np.load(X_VAL_PATH)
y_val_check   = np.load(Y_VAL_PATH)

print("\n📐  Array shapes:")
print(f"   X_train : {X_train_check.shape}   dtype={X_train_check.dtype}")
print(f"   y_train : {y_train_check.shape}   dtype={y_train_check.dtype}")
print(f"   X_val   : {X_val_check.shape}   dtype={X_val_check.dtype}")
print(f"   y_val   : {y_val_check.shape}   dtype={y_val_check.dtype}")

print("\n📊  Class distribution:")
print(f"   {'Class':<5s} {'Name':<22s} {'Train':>10s} {'Train%':>7s}  {'Val':>8s} {'Val%':>7s}")
for cls in sorted(LABEL_CLASS_NAMES):
    tr_cnt = (y_train_check == cls).sum()
    vl_cnt = (y_val_check   == cls).sum()
    tr_pct = tr_cnt / len(y_train_check) * 100
    vl_pct = vl_cnt / len(y_val_check)   * 100
    print(f"   {cls:<5d} {LABEL_CLASS_NAMES[cls]:<22s} {tr_cnt:>10,} {tr_pct:>6.2f}%  {vl_cnt:>8,} {vl_pct:>6.2f}%")

print("\n✅  Sanity checks:")
print(f"   NaN in X_train : {np.isnan(X_train_check).any()}  (should be False)")
print(f"   NaN in X_val   : {np.isnan(X_val_check).any()}  (should be False)")
print(f"   Inf in X_train : {np.isinf(X_train_check).any()}  (should be False)")
print(f"   Feature count  : {X_train_check.shape[1]}")

print("\n📦  Output files:")
for p in sorted(PROCESSED_DIR.iterdir()):
    print(f"   {p.name:<40s}  {p.stat().st_size/1024**2:.1f} MB")

print("\n" + "=" * 70)
print("✅  Phase 1.2 complete — tensors ready for Phase 2 model training")
print("=" * 70)

🔬  FINAL VERIFICATION

📐  Array shapes:
   X_train : (6689792, 48)   dtype=float32
   y_train : (6689792,)   dtype=int8
   X_val   : (836224, 48)   dtype=float32
   y_val   : (836224,)   dtype=int8

📊  Class distribution:
   Class Name                        Train  Train%       Val    Val%
   0     Normal                    400,000   5.98%    50,000   5.98%
   1     Reconnaissance            400,000   5.98%    50,000   5.98%
   2     Volumetric_Attack         400,000   5.98%    50,000   5.98%
   3     C2_Botnet               1,600,000  23.92%   200,000  23.92%
   4     Exploit_and_Theft       3,889,792  58.15%   486,224  58.15%

✅  Sanity checks:
   NaN in X_train : False  (should be False)
   NaN in X_val   : False  (should be False)
   Inf in X_train : False  (should be False)
   Feature count  : 48

📦  Output files:
   subsample.parquet                         251.2 MB
   train_meta.parquet                        83.4 MB
   val_meta.parquet                          10.4 MB
   X_test

In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# CELL 13 | Git Commit
# ─────────────────────────────────────────────────────────────────────────────
import subprocess

NOTEBOOK_DIR = Path.cwd()
MAIN_DIR     = NOTEBOOK_DIR.parent
REPO_ROOT    = MAIN_DIR.parent

def git(args: list, cwd=REPO_ROOT):
    result = subprocess.run(
        ["git"] + args,
        cwd            = str(cwd),
        capture_output = True,
        text           = True,
    )
    if result.stdout.strip():
        print(result.stdout.strip())
    if result.stderr.strip() and result.returncode != 0:
        print("STDERR:", result.stderr.strip())
    return result.returncode

print("📌 Staging files...")
files_to_stage = [
    "main_folder/Phase_1/Phase_1_2_Preprocessing_Normalization.ipynb",
    "main_folder/artifacts/preprocessors.pkl",
    "main_folder/artifacts/phase1_2_feature_names.json",
    "main_folder/Phase_1_1_Universal_Schema_Alignment.md",
]

for f in files_to_stage:
    rc = git(["add", f])
    status = "✅" if rc == 0 else "⚠️ "
    print(f"  {status} staged: {f.split('/')[-1]}")

print("\n📌 Committing...")
commit_msg = (
    "Phase_1_2: Preprocessing & Normalization — "
    "stratified subsample, 80/10/10 split, sentinel imputation, "
    "log1p+StandardScaler, OHE, port-freq encoding; "
    "X_train/X_val tensors + preprocessors.pkl exported"
)
git(["commit", "-m", commit_msg])

print("\n📌 Pushing to remote...")
git(["push"])

print("\n✅ Phase 1.2 committed and pushed.")

📌 Staging files...
  ✅ staged: Phase_1_2_Preprocessing_Normalization.ipynb
STDERR: The following paths are ignored by one of your .gitignore files:
main_folder/artifacts/preprocessors.pkl
hint: Use -f if you really want to add them.
hint: Disable this message with "git config set advice.addIgnoredFile false"
  ⚠️  staged: preprocessors.pkl
  ✅ staged: phase1_2_feature_names.json
  ✅ staged: Phase_1_1_Universal_Schema_Alignment.md

📌 Committing...


[main 66fca55] Phase_1_2: Preprocessing & Normalization â€” stratified subsample, 80/10/10 split, sentinel imputation, log1p+StandardScaler, OHE, port-freq encoding; X_train/X_val tensors + preprocessors.pkl exported
 2 files changed, 398 insertions(+), 140 deletions(-)

📌 Pushing to remote...

✅ Phase 1.2 committed and pushed.


: 